In [83]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
#--------------------------------------------------------------------------------------
#Definerer noen konstanter som ikke forandrer seg i løpet av gjennomkjøringene
#W_k er en dxd matrise i lag k kalt en vekter
#b_k er en d-vektor kalt biaset til lag k
#Y er en dxn matrise med alle bildene. n er antall bilder
#omega er en d-vektor som er en parameter
#my er en skalar som er en parameter
#U er en informasjonsvektor med [[[W_0,b_0],[W_1,b_1],...[W_K-1,b_K-1]],omega,my]. K er antall lag
#Z er prosjisering
#c er fasiten
h = 0.1 #skrittlengde

#--------------------------------------------------------------------------------------
#definerer noen små hjelpefunksjoner som brukes.

#@jit(nopython=True)
def objektfunksjon(Z, C):
    J = 0
    for i in range(len(Z)):
        J += 0.5*np.abs(Z[i]-C[i])**2
    return J

#@jit(nopython=True)
def sigma(X):
    return np.tanh(X)

#@jit(nopython = True)
def sigmader(X):
    return -np.tanh(x)**2+1
    
#@jit(nopython=True)
def etta(X):
    return 0.5*(1+np.tanh(X/2))

#@jit(nopython = True)
def ettader(X):
    return (1/4)*(-(np.tanh(X/2))**2+1)

def packU(W,b,omega,my,K): #Ide for oppbevaring av variable i en vektor U m/ pack/unpack funksjoner
    U=np.array([[[W[r],b[r]] for r in range(0,K)],omega,my])
    return U
    
def unpackU(U,K):#returnerer
    W=[]
    b=[]
    for k in range(0,K):
        W.append(U[0][k][0])
        b.append(U[0][k][1])
    W=np.array(W)
    b=np.array(b)
    return W,b,U[1],U[2]

#------------------------------------------------------------------------------------
#deriverte funksjoner:
#@jit(nopython = True)
def dJdmy(Y, omega, my, Z, C):
    print(Z)
    print(C)
    return ettader(omega.dot(Y.transpose())+(Z-C).dot(my*np.ones(len(Y)).transpose()))#+my*np.ones(len(Y))).transpose().dot((Z-C)) #Må også her sjekke om len(Y[0]) eller len(Y)

#@jit(nopython = True)
def dJdomega(Y,omega, my, Z, C):
    temp=np.array([np.ones(len(Y)) for k in range(K)])
    return Y.dot(((Z-C))*ettader(Y.transpose().dot(omega)+my*temp))#my må kanskje ganges med enhetsmatrisen

#@jit(nopython = True)
def P_K(Y, omega, my, Z, C):
    print(Z)
    print(C)
    return ((Z-C)*(ettader(omega.dot(Y.transpose())+my*np.ones(len(Y))))).transpose().dot(omega)
    #omega.dot(((Z-C)*ettader(omega.dot(Y.transpose())+my*np.ones(len(Y)))))

#@jit(nopython = True)
def P_kmin1(P_k,W_kmin1,Y_kmin1, b_kmin1): 
    return P_k +h*W_kmin1.transpose().dot((sigmader(W_kmin1.dot(Y_kmin1)+b_kmin1)*P_k))

#@jit(nopython = True)
def dJdW(P_kplus1, W, Y, b):
    return h*(P_kplus1*sigmader(W.dot(Y)+b)).dot(Y.transpose())

#@jit(nopython = True)
def dJdb(P_kplus1, W,Y,b):
    return h*(P_kplus1*sigmader(W.dot(Y)+b))*np.ones(len(Y)) #Usikker på denne lengden, men men men
#------------------------------------------------------------------------------------
#Flere hjelpefunksjoner, men disse er litt mer diffuse:
#@jit(nopython = True)
def bildetransformasjon(Y, W, b): #transformerer bildevektorene
    return Y + h*sigma(Y.dot(W)+b)

#@jit(nopython = True)
def prosjisering(Y,omega,my): #prosjiserer bildematrisen, sånn at vi får verdier til å avgjøre om positiv eller negativ
    print(omega.dot(Y.transpose()))
    return etta(omega.dot(Y.transpose())+my*np.ones(len(Y))) #Merk Y[0], eller Y i len()???
    #Dette er altså Z
    
#@jit(nopython=True)
def optimeringsfunk(U,Y,C, tau,K):#Minimerer J
    return U - tau*nabJ(U, Y, C)

#@jit(nopython=True)
def optimeringsfunkiter(U,tau,K,tol):#Minimerer J fram til en toleranse eller maksiter er oppnådd
    error=1
    maxiter=200
    while(error>tol and maxiter<iter):
        oldU=U
        newU=optimeringsfunk(U,tau,K)
        error=objektfunksjon(Z,C)
    return newU
#------------------------------------------------------------------------------------------------
#Nablaoperator
#@jit(nopython=True)
def finnP_k(Y, omega, my, Z, C): #Lager en liste med alle P_k
    P_k = []
    p_k = P_K(Y, omega, my, Z, C)
    P_k.append(p_k)
    for i in range(K-2):
        p_k = P_kmin1(p_k, W[-i-2], Y[-i-2], b[-i-2])
        P_k.append(p_k)
    P_k.reverse()
    return P_k

def nabJ(U,Y,C): #Kanskje fette unødvendig å bruke U her, for så å unnpacke denne shaiten
    W, b , omega, my = unpackU(U,K)
    Z = prosjisering(Y, omega, my)
    dJdW1 = []
    dJdb1 = []
    dJomega = dJdomega(Y, omega, my, Z, C) #???
    dJmy = dJdmy(Y, omega, my, Z, C) #???
    
    P_k = finnP_k(Y, omega, my, Z, C)
    for i in range(0, K-1):
        dJdW1.append(dJdW(P_k[i+1], W, Y, b))
        dJdb1.append(dJdb(P_k[i+1],W,Y,b))
    U = packU(dJW1,dJdb1,djomega,djmy,K)
    return U
#------------------------------------------------------------------------------------------------------


W=np.array([[[1,1,1],[1,1,1],[1,1,1]],[[1,1,1],[1,1,1],[1,1,1]]])
b = np.array([[2,2,2],[2,2,2]])
omega = np.array([3,3,3])
my = 4
K = 2
test=packU(W,b,omega, my,K)

W2,b2,omega2,my2=unpackU(test,K)




In [94]:
#import spirals.py
from spirals import get_data_spiral_2d
import random

def algoritme():#Merk!!! Y må starte som kun et datapunkt!!
    datapkt=2 #Definerer hvor mange bilder vi tar inn
    K=20 #Definerer K
    tau=0.01 #Definerer tau
    omega= np.ones(datapkt)#definerer omega np.random.uniform(0, 1, datapkt)
    my=1 #definerer my
    W=np.array([[np.random.uniform(0,1,datapkt) for i in range(datapkt)] for j in range(K)]) #Lager vekter med tilfeldige verdier
    b=[np.random.uniform(0, 1, datapkt) for i in range(K)] #Lager bias med tilfeldige verdier
    Y,C=get_data_spiral_2d(datapkt) #henter inn data Y og fasit C
    nyC=[]
    for i in C:
        if i:
            nyC.append(1)
        else:
            nyC.append(0)
    C=nyC
    #Printer ut variable for å se hva tinga er
    print("d:",10,"K:",K,"tau:",tau,"my:",my)
    print("-----------------------------------")
    print("Y:",Y)
    print("-----------------------------------")
    print("omega:",omega)
    print("-----------------------------------")
    print("W:",W)
    print("-----------------------------------")
    print("b:",b,"Ytterlengde:",len(b),"Innerlengde:",len(b[0]))
    print("-----------------------------------")
    print("C:",C)
    print("Nå er vi i algoritmen")
    print("-----------------------------------")
    iter=0
    while(iter<10):
        for i in range(K):
            Y=bildetransformasjon(Y,W[i],b[i])
        Z=prosjisering(Y,omega,my)
        PK=P_K(Y,omega,my,Z,C)
        U=packU(W,b,omega,my,K)
        U=optimeringsfunk(U,Y,C,tau,K)
        W,b,omega,my=unpackU(U,K)
        iter+=1
    U=packU(W,b,omega,my,K)
    return U

print(algoritme())





d: 10 K: 20 tau: 0.01 my: 1
-----------------------------------
Y: [[ 0.07696407 -0.14652738]
 [-0.00727153  0.04761499]]
-----------------------------------
omega: [1. 1.]
-----------------------------------
W: [[[0.76060833 0.12994279]
  [0.31641994 0.63614703]]

 [[0.56366547 0.24727184]
  [0.59359016 0.36703727]]

 [[0.93574597 0.35557661]
  [0.02341952 0.26182926]]

 [[0.81800687 0.4904824 ]
  [0.2379213  0.62839016]]

 [[0.15352403 0.39935795]
  [0.81014633 0.78676106]]

 [[0.97436021 0.33937015]
  [0.45584548 0.94545815]]

 [[0.32002933 0.7932088 ]
  [0.10565403 0.20528767]]

 [[0.99365388 0.87502697]
  [0.18342128 0.61309736]]

 [[0.28478688 0.76172255]
  [0.55428966 0.0454589 ]]

 [[0.09382258 0.52336331]
  [0.29060514 0.15138822]]

 [[0.12382619 0.63889846]
  [0.56057248 0.65821462]]

 [[0.91355409 0.97374698]
  [0.35161143 0.36398872]]

 [[0.12833336 0.56822465]
  [0.97386869 0.89052845]]

 [[0.56434732 0.93596737]
  [0.59843001 0.46122814]]

 [[0.27160306 0.88060929]
  [0.4

IndexError: list index out of range